In [ ]:
import requests
import json
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#NLTK
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [ ]:
from wordcloud import WordCloud

## Llamada a la API para obtener todos los dialogos.

In [ ]:
url = "http://127.0.0.1:5000/"
#GET

endpoint_user = "users"
endopoint_message= "messages"
endopoint_user_message= "<episode>/<name>"

In [ ]:
mess = requests.get(url + endopoint_message).json()
mess[:5]

### Hacemos un dataframe con ello.

In [ ]:
data = pd.DataFrame(mess)

In [ ]:
data.head()

### Aplicamos Stop Words de NLTK para deshacernos de las palabras neutras.

In [ ]:
nltk.download('stopwords')

In [ ]:
#stopwords.words('english')

In [ ]:
data["Line"] = data["Line"].astype(str)

data.head()

In [ ]:
data.Line = data.Line.apply(lambda x: x.split(" "))
data.head()

In [ ]:
def stop_words (lista):
    stop_words = set(stopwords.words('english'))
    nueva_lista = []
    for string in lista:
        if string.lower() not in stop_words:
            nueva_lista.append(string)
    return " ".join(nueva_lista)

In [ ]:
data.Line = data.Line.apply(stop_words)
data.head()

## Realizamos Sentiment Analysis con Vader Lexicon de NLTK 

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
def sentimentAnalysis(sentence):
    sia = SentimentIntensityAnalyzer()
    polarity = sia.polarity_scores(sentence)
    pol = polarity['compound']
    return pol

In [ ]:
data['sentiment_compound'] = data.Line.apply(sentimentAnalysis)
data.head()

In [ ]:
resumen = data.groupby(['Character'])['sentiment_compound'].mean().sort_values()
#resumen

In [ ]:
tail_5 = resumen[:5]
tail_5

### Darryl es quien obtiene la peor puntuación al analizar los 3 primeros capitulos de la temporada 10.

In [ ]:
tail_5.plot(kind="bar", color= "lightcoral")

In [ ]:
top_5 = resumen[-5:]
top_5

### Townsfolf es quien obtiene la mejor puntuación al analizar los 3 primeros capitulos de la temporada 10. Entendemos que el Speaker es el narrador de los capitulos.

In [ ]:
top_5.plot(kind="bar", color = "lightgreen")

## Creamos dataframe solo con los 4 protagonistas.

In [ ]:
resumen = pd.DataFrame(resumen)
#resumen

In [ ]:
resumen['protas'] = resumen.index

In [ ]:
resumen2 = resumen[(resumen["protas"]== "Cartman")| (resumen["protas"]== "Kenny") | (resumen["protas"]== "Kyle") | (resumen["protas"]== "Stan") ]

In [ ]:
resumen2

### Todos los protagonistas obtienen un analisis neutro, todos los valores muy ceca del cero. 

In [ ]:
sns.barplot(
    x = resumen2["protas"],
    y = resumen2["sentiment_compound"],
    palette= "PRGn"
)

### Analisis con WordCloud para ver cuales son las palabras más utilizadas.

In [ ]:
df_protas = data[(data["Character"]== "Cartman")| (data["Character"]== "Kenny") | (data["Character"]== "Kyle") | (data["Character"]== "Stan") ]

In [ ]:
df_protas.head()

In [ ]:
def generateWordCloud(data,title):
    wordcloud = WordCloud(background_color='black',
        colormap= 'Set2',
        max_words=500,
        max_font_size=50, 
        scale=5,
        random_state=3).generate(str(data))


    wordcloud.recolor(random_state=1)
    plt.figure(figsize=(15,15))
    plt.title(title, fontsize=20,color='black')
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [ ]:
 generateWordCloud(df_protas['Line'],"Word Cloud Season 10.")